# **Telecom Churn Data | ETL**

*Authors:*
- *Myroslava Sánchez Andrade A01730712*
- *Karen Rugerio Armenta A01733228*
- *José Antonio Bobadilla García A01734433*
- *Alejandro Castro Reus A01731065*

*Creation date: 17/08/2022*

*Last updated: 11/09/2022*

---

## **Extract**
The dataset used for this project is **[telecom_churn_me.csv](https://www.kaggle.com/datasets/mark18vi/telecom-churn-data?resource=download)**, downloaded from the plataform Kaggle.
<br>This dataset of a telecommunications company contains the costumers' account information and whether the customers left or not within the last month.


In [2]:
# REQUIRED LIBRARIES
# !pip install pandas numpy matplotlib statsmodels sklearn scipy

In [3]:
# RUN ONLY FOR GOOGLE COLAB

# from google.colab import drive

# drive.mount("path")  

# %cd "path"

In [4]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import preprocessing
from scipy import stats



In [5]:
# Reading data via Pandas from CSV
telco_customers_data = pd.read_csv('../../../data/telecom_churn_me.csv')
telco_customers_data

,Unnamed: 0,PTY_PROFILE_SUB_TYPE,SOCIO_ECONOMIC_SEGMENT,PARTY_NATIONALITY,PARTY_GENDER_CD,TARGET,YEAR_JOINED,CURRENT_YEAR,BILL_AMOUNT,PAID_AMOUNT,...,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,DATA_OUT_BNDL,DATA_USG_PAYG,COMPLAINTS,Years_stayed
0,0,Residential,EMIRATI,United Arab Emirates,M,0,1994,2019,931.208938,812.175000,...,35.850,34.015,72.075,141.840,56.115,11944.079102,0.0,0.0,0,25
1,1,Prestige,EMIRATI,United Arab Emirates,M,0,1994,2019,431.082618,486.500000,...,10.595,7.715,11.750,5.110,0.000,9903.157715,0.0,0.0,0,25
2,2,Residential,EMIRATI,United Arab Emirates,M,0,1994,2019,50.619644,52.815000,...,0.000,0.000,0.000,0.000,0.000,0.102539,0.0,0.0,0,25
3,3,Prestige,EMIRATI,United Arab Emirates,M,0,1994,2019,399.710034,422.235000,...,158.500,2.670,15.965,0.000,0.000,3600.322266,0.0,0.0,0,25
4,4,Residential,EMIRATI,United Arab Emirates,M,0,1994,2019,612.665844,825.888333,...,186.050,17.515,28.685,3.235,4.475,3852.026367,0.0,0.0,0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140599,1140610,Residential,EMIRATI,United Arab Emirates,M,0,2017,2019,297.752650,313.950000,...,0.000,0.000,0.000,0.000,0.000,307945.957031,0.0,0.0,0,2
1140600,1140611,Residential,YOUTH,United Arab Emirates,M,0,2017,2019,160.663773,178.500000,...,0.000,0.000,0.000,0.000,0.000,22647.873535,0.0,0.0,0,2
1140601,1140612,Consumer via Retailer,EXPATS,Comoros,M,0,2017,2019,570.147016,642.911667,...,64.990,3.660,10.050,0.000,0.000,17582.867188,0.0,0.0,0,2
1140602,1140613,Residential,EXPATS,Philippines,M,0,2017,2019,452.736799,525.413333,...,102.075,54.065,7.980,5.350,0.065,3015.338867,0.0,0.0,0,2


#### ***Verifying structure and content***

In [6]:
# Validating the information of each column => There are no null values in the whole DF, there are multiple objects.
telco_customers_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140604 entries, 0 to 1140603
Data columns (total 28 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Unnamed: 0                 1140604 non-null  int64  
 1   PTY_PROFILE_SUB_TYPE       1140604 non-null  object 
 2   SOCIO_ECONOMIC_SEGMENT     1140604 non-null  object 
 3   PARTY_NATIONALITY          1140604 non-null  object 
 4   PARTY_GENDER_CD            1140604 non-null  object 
 5   TARGET                     1140604 non-null  int64  
 6   YEAR_JOINED                1140604 non-null  int64  
 7   CURRENT_YEAR               1140604 non-null  int64  
 8   BILL_AMOUNT                1140604 non-null  float64
 9   PAID_AMOUNT                1140604 non-null  float64
 10  PAYMENT_TRANSACTIONS       1140604 non-null  int64  
 11  PARTY_REV                  1140604 non-null  float64
 12  PREPAID_LINES              1140604 non-null  int64  
 13  POSTPAID_LIN

---
## **Transform**

### ***Column analysis***

#### *Column valuation*

Dropping the columns that have at least 65% null values

In [7]:
telco_customers_data = telco_customers_data.dropna(thresh = (telco_customers_data.shape[0] * 0.65) , axis = 1)

Storing the columns that are full of unique values

In [8]:
# Storing the number of unique values of each column
no_unique_values = telco_customers_data.nunique().to_frame()

# Storing the name of columns that are full of unique values (id)
drop_columns = no_unique_values[no_unique_values == telco_customers_data.shape[0]]
drop_columns = drop_columns.dropna()

drop_column_names = []
drop_column_names.append(drop_columns.index[0])
drop_column_names

['Unnamed: 0']

Storing the columns that are full (or almost full) of the same values

In [9]:
# Calculating the percentiles of each column
data_description = telco_customers_data.describe()
data_description = data_description.drop(['count', 'mean', 'std', 'min', 'max'], axis = 0)
data_description

,Unnamed: 0,TARGET,YEAR_JOINED,CURRENT_YEAR,BILL_AMOUNT,PAID_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,POSTPAID_LINES,...,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,DATA_OUT_BNDL,DATA_USG_PAYG,COMPLAINTS,Years_stayed
25%,285154.75,0.0,2013.0,2019.0,174.137757,181.666667,1.0,423.187917,0.0,1.0,...,0.425,0.350,0.015,0.000,0.00,708.101562,0.0,0.0,0.0,2.0
50%,570306.50,0.0,2016.0,2019.0,290.723940,300.729167,1.0,834.713333,1.0,2.0,...,29.445,7.160,10.025,2.175,0.00,4394.218506,0.0,0.0,0.0,3.0
75%,855461.25,0.0,2017.0,2019.0,460.977100,476.423333,2.0,1553.675000,3.0,3.0,...,141.895,22.035,39.260,54.090,1.71,9955.910278,0.0,0.0,0.0,6.0


In [10]:
# Storing the difference column by column of the percentiles
data_description = data_description.diff()
data_description

,Unnamed: 0,TARGET,YEAR_JOINED,CURRENT_YEAR,BILL_AMOUNT,PAID_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,POSTPAID_LINES,...,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,DATA_OUT_BNDL,DATA_USG_PAYG,COMPLAINTS,Years_stayed
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,285151.75,0.0,3.0,0.0,116.586183,119.062500,0.0,411.525417,1.0,1.0,...,29.02,6.810,10.010,2.175,0.00,3686.116943,0.0,0.0,0.0,1.0
75%,285154.75,0.0,1.0,0.0,170.253160,175.694167,1.0,718.961667,2.0,1.0,...,112.45,14.875,29.235,51.915,1.71,5561.691772,0.0,0.0,0.0,3.0


In [11]:
# If the difference is 0 in percentile 50 and 75, it means that the column has no variation in its values (columns full of the same value)
percentiles = data_description[1:2] == 0.0
percentiles = percentiles.append(data_description[2:3] == 0.0)

for col in data_description.columns:
    if percentiles[col].all() == False:
        percentiles = percentiles.drop(col, axis=1)

print(percentiles)
drop_column_names.extend(list(percentiles.columns))
drop_column_names

     TARGET  CURRENT_YEAR  DATA_OUT_BNDL  DATA_USG_PAYG  COMPLAINTS
50%    True          True           True           True        True
75%    True          True           True           True        True


C:\Users\myros\AppData\Local\Temp\ipykernel_22876\4217765842.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  percentiles = percentiles.append(data_description[2:3] == 0.0)


['Unnamed: 0',
 'TARGET',
 'CURRENT_YEAR',
 'DATA_OUT_BNDL',
 'DATA_USG_PAYG',
 'COMPLAINTS']

Dropping the columns stored in steps before

In [12]:
# Removing the target from the columns to drop
y = 'TARGET'
drop_column_names.remove(y)

# Dropping the columns to drop
telco_customers_data = telco_customers_data.drop(drop_column_names, axis=1)
telco_customers_data

,PTY_PROFILE_SUB_TYPE,SOCIO_ECONOMIC_SEGMENT,PARTY_NATIONALITY,PARTY_GENDER_CD,TARGET,YEAR_JOINED,BILL_AMOUNT,PAID_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,...,LINE_REV,STATUS,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,Years_stayed
0,Residential,EMIRATI,United Arab Emirates,M,0,1994,931.208938,812.175000,1,5968.700000,...,945.040000,ACTIVE,1004.070,35.850,34.015,72.075,141.840,56.115,11944.079102,25
1,Prestige,EMIRATI,United Arab Emirates,M,0,1994,431.082618,486.500000,1,6245.141667,...,493.815000,ACTIVE,159.050,10.595,7.715,11.750,5.110,0.000,9903.157715,25
2,Residential,EMIRATI,United Arab Emirates,M,0,1994,50.619644,52.815000,1,1666.488333,...,50.300000,ACTIVE,0.000,0.000,0.000,0.000,0.000,0.000,0.102539,25
3,Prestige,EMIRATI,United Arab Emirates,M,0,1994,399.710034,422.235000,1,2522.008333,...,406.586667,ACTIVE,288.805,158.500,2.670,15.965,0.000,0.000,3600.322266,25
4,Residential,EMIRATI,United Arab Emirates,M,0,1994,612.665844,825.888333,1,1219.961667,...,751.185000,ACTIVE,209.760,186.050,17.515,28.685,3.235,4.475,3852.026367,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140599,Residential,EMIRATI,United Arab Emirates,M,0,2017,297.752650,313.950000,1,2418.486667,...,303.166667,ACTIVE,0.000,0.000,0.000,0.000,0.000,0.000,307945.957031,2
1140600,Residential,YOUTH,United Arab Emirates,M,0,2017,160.663773,178.500000,1,454.116667,...,170.000000,ACTIVE,0.000,0.000,0.000,0.000,0.000,0.000,22647.873535,2
1140601,Consumer via Retailer,EXPATS,Comoros,M,0,2017,570.147016,642.911667,1,615.866667,...,609.630000,ACTIVE,154.150,64.990,3.660,10.050,0.000,0.000,17582.867188,2
1140602,Residential,EXPATS,Philippines,M,0,2017,452.736799,525.413333,2,735.645000,...,414.840000,ACTIVE,218.805,102.075,54.065,7.980,5.350,0.065,3015.338867,2


#### *One hot encoding*

In [13]:
# Identifying the categorical columns
no_unique_values = telco_customers_data.drop(y, axis=1).nunique().to_frame()
categorical_columns = no_unique_values[no_unique_values <= telco_customers_data.shape[0]*0.02]
categorical_columns = categorical_columns.dropna()
categorical_columns = list(categorical_columns.index)
categorical_columns

['PTY_PROFILE_SUB_TYPE',
 'SOCIO_ECONOMIC_SEGMENT',
 'PARTY_NATIONALITY',
 'PARTY_GENDER_CD',
 'YEAR_JOINED',
 'PAYMENT_TRANSACTIONS',
 'PREPAID_LINES',
 'POSTPAID_LINES',
 'OTHER_LINES',
 'STATUS',
 'Years_stayed']

In [14]:
# Creating a label encoder object
le = preprocessing.LabelEncoder()
# Categorizing the column values with numbers
categorical_columns = telco_customers_data[categorical_columns].apply(le.fit_transform)
categorical_columns

,PTY_PROFILE_SUB_TYPE,SOCIO_ECONOMIC_SEGMENT,PARTY_NATIONALITY,PARTY_GENDER_CD,YEAR_JOINED,PAYMENT_TRANSACTIONS,PREPAID_LINES,POSTPAID_LINES,OTHER_LINES,STATUS,Years_stayed
0,2,0,179,1,0,1,2,5,2,0,25
1,1,0,179,1,0,1,6,3,2,0,25
2,2,0,179,1,0,1,2,2,1,0,25
3,1,0,179,1,0,1,3,3,3,0,25
4,2,0,179,1,0,1,0,1,1,0,25
...,...,...,...,...,...,...,...,...,...,...,...
1140599,2,0,179,1,23,1,5,3,3,0,2
1140600,2,2,179,1,23,1,0,0,1,0,2
1140601,0,1,39,1,23,1,1,0,0,0,2
1140602,2,1,136,1,23,2,1,1,0,0,2


In [15]:
# Subsituting the numerical categorical columns in the dataset
telco_customers_data[categorical_columns.columns] = categorical_columns
telco_customers_data

,PTY_PROFILE_SUB_TYPE,SOCIO_ECONOMIC_SEGMENT,PARTY_NATIONALITY,PARTY_GENDER_CD,TARGET,YEAR_JOINED,BILL_AMOUNT,PAID_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,...,LINE_REV,STATUS,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,Years_stayed
0,2,0,179,1,0,0,931.208938,812.175000,1,5968.700000,...,945.040000,0,1004.070,35.850,34.015,72.075,141.840,56.115,11944.079102,25
1,1,0,179,1,0,0,431.082618,486.500000,1,6245.141667,...,493.815000,0,159.050,10.595,7.715,11.750,5.110,0.000,9903.157715,25
2,2,0,179,1,0,0,50.619644,52.815000,1,1666.488333,...,50.300000,0,0.000,0.000,0.000,0.000,0.000,0.000,0.102539,25
3,1,0,179,1,0,0,399.710034,422.235000,1,2522.008333,...,406.586667,0,288.805,158.500,2.670,15.965,0.000,0.000,3600.322266,25
4,2,0,179,1,0,0,612.665844,825.888333,1,1219.961667,...,751.185000,0,209.760,186.050,17.515,28.685,3.235,4.475,3852.026367,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140599,2,0,179,1,0,23,297.752650,313.950000,1,2418.486667,...,303.166667,0,0.000,0.000,0.000,0.000,0.000,0.000,307945.957031,2
1140600,2,2,179,1,0,23,160.663773,178.500000,1,454.116667,...,170.000000,0,0.000,0.000,0.000,0.000,0.000,0.000,22647.873535,2
1140601,0,1,39,1,0,23,570.147016,642.911667,1,615.866667,...,609.630000,0,154.150,64.990,3.660,10.050,0.000,0.000,17582.867188,2
1140602,2,1,136,1,0,23,452.736799,525.413333,2,735.645000,...,414.840000,0,218.805,102.075,54.065,7.980,5.350,0.065,3015.338867,2


#### *Multicollinearity*

In [16]:
# VIF dataframe
vif_data = pd.DataFrame()

# Removing the dependent variable
x_variables = telco_customers_data.drop(y, axis=1)
vif_data["x variables"] = x_variables.columns
vif_data

,x variables
0,PTY_PROFILE_SUB_TYPE
1,SOCIO_ECONOMIC_SEGMENT
2,PARTY_NATIONALITY
3,PARTY_GENDER_CD
4,YEAR_JOINED
5,BILL_AMOUNT
6,PAID_AMOUNT
7,PAYMENT_TRANSACTIONS
8,PARTY_REV
9,PREPAID_LINES


In [18]:
# Calculating the vif of the columns and dropping the high multicollinearity
def calculate_vif(vif_data, x_variables):
    vif_data['VIF'] = [variance_inflation_factor(x_variables.values, i) for i in range(len(x_variables.columns))]
    while vif_data['VIF'].max() > 5:
        max_index = vif_data['VIF'].idxmax()
        delete_column = vif_data['x variables'].iloc[max_index]
        x_variables = x_variables.drop(columns=[delete_column], axis=1)
        vif_data = vif_data.drop(index=max_index, axis=0)
        vif_data['VIF'] = [variance_inflation_factor(x_variables.values, i) for i in range(len(x_variables.columns))]
        vif_data.reset_index(inplace=True, drop=True)
    return vif_data
    

# Storing the columns with no multicolinearity
filtered_vif_data = calculate_vif(vif_data, x_variables)
filtered_vif_data

,x variables,VIF
0,SOCIO_ECONOMIC_SEGMENT,2.467935
1,PARTY_GENDER_CD,3.912827
2,BILL_AMOUNT,2.470802
3,PAYMENT_TRANSACTIONS,3.629384
4,PARTY_REV,2.527949
5,PREPAID_LINES,2.179423
6,POSTPAID_LINES,3.199450
7,OTHER_LINES,3.724654
8,STATUS,1.083224
9,MOUS_TO_LOCAL_MOBILES,1.366994


Multicollinearity occurs when two or more independent variables have high correlation themselves and it might cause an unreliable estimation, thus, these variables must be detected and discarded.

For the detection of multicollinearity, the **Variance Inflation Factor (VIF)** technique was used. This method regress each independent variable against all others. The VIF is calculated: $VIF = {1\over 1 - R^2}$, where $R^2$ is the coefficient of determination in linear regression. A higher VIF denotates a strong collinearity. Generally, a VIF above 5 indicates a high multicollinearity. 

In [22]:
filtered_columns = list(filtered_vif_data['x variables'])
filtered_columns.append(y)

telco_customers_data = telco_customers_data[filtered_columns]
telco_customers_data

,SOCIO_ECONOMIC_SEGMENT,PARTY_GENDER_CD,BILL_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,POSTPAID_LINES,OTHER_LINES,STATUS,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,Years_stayed,TARGET
0,0,1,931.208938,1,5968.700000,2,5,2,0,1004.070,35.850,34.015,72.075,141.840,56.115,11944.079102,25,0
1,0,1,431.082618,1,6245.141667,6,3,2,0,159.050,10.595,7.715,11.750,5.110,0.000,9903.157715,25,0
2,0,1,50.619644,1,1666.488333,2,2,1,0,0.000,0.000,0.000,0.000,0.000,0.000,0.102539,25,0
3,0,1,399.710034,1,2522.008333,3,3,3,0,288.805,158.500,2.670,15.965,0.000,0.000,3600.322266,25,0
4,0,1,612.665844,1,1219.961667,0,1,1,0,209.760,186.050,17.515,28.685,3.235,4.475,3852.026367,25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140599,0,1,297.752650,1,2418.486667,5,3,3,0,0.000,0.000,0.000,0.000,0.000,0.000,307945.957031,2,0
1140600,2,1,160.663773,1,454.116667,0,0,1,0,0.000,0.000,0.000,0.000,0.000,0.000,22647.873535,2,0
1140601,1,1,570.147016,1,615.866667,1,0,0,0,154.150,64.990,3.660,10.050,0.000,0.000,17582.867188,2,0
1140602,1,1,452.736799,2,735.645000,1,1,0,0,218.805,102.075,54.065,7.980,5.350,0.065,3015.338867,2,0


#### *Outlier management*

In [ ]:
z = np.abs(stats.zscore())
print(z)

#### *Standardization*

### ***Row analysis***

#### *Row valuation*

### ***Imputation***

### ***Data preparation***

In [2]:
# Train, validation, test

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\myros\anaconda3\envs\Statistics\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\myros\anaconda3\envs\Statistics\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\myros\anaconda3\envs\Statistics\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\myros\anaconda3\envs\Statistics\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetErro

---
## **Load**